### 读取数据

In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)

data = pd.read_csv('./data/data.csv')
log = pd.read_csv('./data/log.csv')

In [2]:
pd.set_option('display.max_columns',1000)

In [3]:
# user_prop = list(data.columns)
# user_prop.pop()
# user_prop.pop()

# normal_value = []

# def find_range(list):
#     for x in list:
#         top_border = data[x].quantile(0.98)
#         bottom_border = data[x].quantile(0.02)
#         normal_value.append(dict({
#             'top': top_border,
#             'bottom': bottom_border
#         }))
        
# find_range(user_prop)

# data_range = pd.DataFrame()
# def in_range(list):
#     for i, x in enumerate(list):
#         stri = 'IN-' + str(i + 1)
#         data_range[stri] = data[x].apply(lambda x: int(x > normal_value[i]['top'] or x < normal_value[i]['bottom']))
        
# in_range(user_prop)
# data_range['OUT-NUM'] = data_range.apply(lambda x: x.sum(), axis=1)
# # data_range = data_range['OUT-NUM']
# # 回头再看
# # data = pd.concat([data, data_range], axis=1, copy=False)
# data.head()

### 构造特征

### 1. 操作h5的次数的特征（为了频率特征，后面舍弃）

In [4]:
data_h5 = log[log['TCH_TYP'] == 2]
data_h5['H5_NUM'] = 1
data_h5 = data_h5.groupby('USRID', as_index=False)['H5_NUM'].sum()
data_h5.head()

/Users/Liulu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,USRID,H5_NUM
0,16,1
1,56,1
2,115,1
3,195,31
4,250,1


### 2. 每个用户操作特征（操作的总数，平均值，最小值和总数）

In [5]:
log['OPR-NUM'] = 1
opr_num = log.groupby('USRID', as_index=False)['OPR-NUM'].sum()
opr_num.head()

,USRID,OPR-NUM
0,2,9
1,3,157
2,4,18
3,5,20
4,7,76


In [6]:
log_eve = log.groupby(['USRID', 'DOM'], as_index=False)['OPR-NUM'].sum()
log_eve = log_eve.groupby('USRID', as_index=False)['OPR-NUM'].agg({
    'OPR-NUM-MAX': np.max,
    'OPR-NUM-MIN': np.min,
    'OPR-NUM-MEA': np.mean,
    'OPR-NUM-STD': np.std
})
log_eve.head()

,USRID,OPR-NUM-MAX,OPR-NUM-MIN,OPR-NUM-MEA,OPR-NUM-STD
0,2,8,1,4.500000,4.949747
1,3,30,6,17.444444,8.427798
2,4,18,18,18.000000,NaN
3,5,10,2,6.666667,4.163332
4,7,32,8,15.200000,9.959920


In [7]:
log_opr = pd.merge(opr_num, log_eve, on='USRID', how='left')
data = pd.merge(data, log_opr, on='USRID', how='left')
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'USRID',
       'FLAG', 'OPR-NUM', 'OPR-NUM-MAX', 'OPR-NUM-MIN', 'OPR-NUM-MEA',
       'OPR-NUM-STD'],
      dtype='object')

### 3.操作h5的频率特征

In [8]:
data_h5 = pd.merge(opr_num, data_h5, on='USRID', how='left').fillna(0)
data_h5['H5_FRQ'] = data_h5['H5_NUM'] / data_h5['OPR-NUM']
data_h5 = data_h5.drop(['OPR-NUM'], axis=1)

In [9]:
data_h5.head()

,USRID,H5_NUM,H5_FRQ
0,2,0.0,0.0
1,3,0.0,0.0
2,4,0.0,0.0
3,5,0.0,0.0
4,7,0.0,0.0


In [10]:
data = pd.merge(data, data_h5, on='USRID', how='left')

In [11]:
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'USRID',
       'FLAG', 'OPR-NUM', 'OPR-NUM-MAX', 'OPR-NUM-MIN', 'OPR-NUM-MEA',
       'OPR-NUM-STD', 'H5_NUM', 'H5_FRQ'],
      dtype='object')

In [12]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,USRID,FLAG,OPR-NUM,OPR-NUM-MAX,OPR-NUM-MIN,OPR-NUM-MEA,OPR-NUM-STD,H5_NUM,H5_FRQ
0,-1.92554,0,-1.26634,1,0,-1.70618,-0.29641,-0.18761,-0.48351,-0.53051,-0.18703,-0.41609,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.42763,-0.30845,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55917,-0.77398,-0.25708,-0.27221,-0.51336,-0.32438,14233,0,468.0,118.0,2.0,58.50,38.026307,0.0,0.0
1,-1.92554,0,-1.26634,1,0,-1.70618,0.37601,-0.10030,-0.16694,-0.31897,0.13488,-0.41609,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.49415,-0.43058,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.50058,-0.77398,-0.25708,-0.37907,-0.52723,-0.32438,790,0,243.0,75.0,4.0,20.25,18.805826,0.0,0.0
2,-1.92554,0,-1.26634,1,0,-1.57176,0.03980,-0.17089,-0.38610,-0.46000,-0.36669,-0.17962,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.57339,-0.45064,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55127,-0.77398,-0.25708,-0.57113,-0.54758,-0.32438,82308,0,195.0,47.0,4.0,13.00,11.420533,0.0,0.0
3,-1.92554,0,-1.26634,1,0,-1.30291,-0.29641,-0.18761,-0.45916,-0.53051,-0.11165,-0.35697,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.45451,-0.17717,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.26154,-0.77398,0.23440,-0.32873,-0.51951,-0.32438,90325,0,126.0,29.0,6.0,14.00,7.810250,0.0,0.0
4,-1.92554,0,-1.26634,1,0,-1.16849,-0.29641,-0.18761,-0.50786,-0.60103,-0.36994,-0.53433,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.57729,-0.45963,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55917,-0.77398,-0.25708,-0.57910,-0.54884,-0.32438,71893,0,4.0,2.0,2.0,2.00,0.000000,0.0,0.0


### 4. 月初、月末点击的次数和频率

In [13]:
def week_1(x):
    if x >= 1 and x <= 4:
        return 1
    else:
        return 0

def week_2(x):
    if x >= 5 and x <= 11:
        return 1
    else:
        return 0
    
def week_3(x):
    if x >= 12 and x <= 18:
        return 1
    else:
        return 0

def week_4(x):
    if x >= 19 and x <= 25:
        return 1
    else:
        return 0

def week_5(x):
    if x >= 26 and x <= 31:
        return 1
    else:
        return 0
    
log['WEK_1'] = log['DOM'].apply(lambda x: week_1(x))
log['WEK_2'] = log['DOM'].apply(lambda x: week_2(x))
log['WEK_3'] = log['DOM'].apply(lambda x: week_3(x))
log['WEK_4'] = log['DOM'].apply(lambda x: week_4(x))
log['WEK_5'] = log['DOM'].apply(lambda x: week_5(x))

In [14]:
data_week1 = pd.DataFrame(log, columns=['USRID', 'WEK_1'])
data_week1 = data_week1.groupby('USRID', as_index=False).sum()
data_week1 = pd.merge(opr_num, data_week1, on=['USRID'], how='left').fillna(0)
data_week1['WEK_1_RTE'] = data_week1['WEK_1'] / data_week1['OPR-NUM']
data_week1 = data_week1.drop(['OPR-NUM'], axis=1)

data_week2 = pd.DataFrame(log, columns=['USRID', 'WEK_2'])
data_week2 = data_week2.groupby('USRID', as_index=False).sum()
data_week2 = pd.merge(opr_num, data_week2, on=['USRID'], how='left').fillna(0)
data_week2['WEK_2_RTE'] = data_week2['WEK_2'] / data_week2['OPR-NUM']
data_week2 = data_week2.drop(['OPR-NUM'], axis=1)

data_sides12 = pd.merge(data_week1, data_week2, on=['USRID'], how='left')

data_week3 = pd.DataFrame(log, columns=['USRID', 'WEK_3'])
data_week3 = data_week3.groupby('USRID', as_index=False).sum()
data_week3 = pd.merge(opr_num, data_week3, on=['USRID'], how='left').fillna(0)
data_week3['WEK_3_RTE'] = data_week3['WEK_3'] / data_week3['OPR-NUM']
data_week3 = data_week3.drop(['OPR-NUM'], axis=1)

data_sides123 = pd.merge(data_sides12, data_week3, on=['USRID'], how='left')

data_week4 = pd.DataFrame(log, columns=['USRID', 'WEK_4'])
data_week4 = data_week4.groupby('USRID', as_index=False).sum()
data_week4 = pd.merge(opr_num, data_week4, on=['USRID'], how='left').fillna(0)
data_week4['WEK_4_RTE'] = data_week4['WEK_4'] / data_week4['OPR-NUM']
data_week4 = data_week4.drop(['OPR-NUM'], axis=1)

data_sides1234 = pd.merge(data_sides123, data_week4, on=['USRID'], how='left')

data_week5 = pd.DataFrame(log, columns=['USRID', 'WEK_5'])
data_week5 = data_week5.groupby('USRID', as_index=False).sum()
data_week5 = pd.merge(opr_num, data_week5, on=['USRID'], how='left').fillna(0)
data_week5['WEK_5_RTE'] = data_week5['WEK_5'] / data_week5['OPR-NUM']
data_week5 = data_week5.drop(['OPR-NUM'], axis=1)

data_sides = pd.merge(data_sides1234, data_week5, on=['USRID'], how='left')

data = pd.merge(data, data_sides, on='USRID', how='left')
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'USRID',
       'FLAG', 'OPR-NUM', 'OPR-NUM-MAX', 'OPR-NUM-MIN', 'OPR-NUM-MEA',
       'OPR-NUM-STD', 'H5_NUM', 'H5_FRQ', 'WEK_1', 'WEK_1_RTE', 'WEK_2',
       'WEK_2_RTE', 'WEK_3', 'WEK_3_RTE', 'WEK_4', 'WEK_4_RTE', 'WEK_5',
       'WEK_5_RTE'],
      dtype='object')

### 5. 优惠券种类总数特征

In [15]:
log['EVT_LBL_1'] = log['EVT_LBL_1'].astype(str)
tic_cls = pd.get_dummies(log['EVT_LBL_1'], prefix='TIC_CLS')
data_tic = pd.concat([log['USRID'], tic_cls], axis=1, copy=False)
tic_sum = data_tic.groupby('USRID', as_index=False).sum()
# 缺失值处理
tic_sum = pd.merge(pd.DataFrame(data['USRID']), tic_sum, on=['USRID'], copy=False, how='left').fillna(0)
data = pd.merge(data, tic_sum, on='USRID', how='left')
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'USRID',
       'FLAG', 'OPR-NUM', 'OPR-NUM-MAX', 'OPR-NUM-MIN', 'OPR-NUM-MEA',
       'OPR-NUM-STD', 'H5_NUM', 'H5_FRQ', 'WEK_1', 'WEK_1_RTE', 'WEK_2',
       'WEK_2_RTE', 'WEK_3', 'WEK_3_RTE', 'WEK_4', 'WEK_4_RTE', 'WEK_5',
       'WEK_5_RTE', 'TIC_CLS_0', 'TIC_CLS_10', 'TIC_CLS_102', 'TIC_CLS_139',
       'TIC_CLS_162', 'TIC_CLS_163', 'TIC_CLS_181', 'TIC_CLS_257',
       'TIC_CLS_259', 'TIC_CLS_326', 'TIC_CLS_359', 'TIC_CLS_372',
       'TIC_CLS_38', 'TIC_CLS_396', 'TIC_CLS_438', 'TIC_CLS_460',
       'TIC_CLS_508', 'TIC_CLS_518', 'TIC_CLS_520', 'TIC_CLS_540',
       'TIC_CLS_604'],
      dtype='object')

In [16]:
log['HOUR'] = log['HOUR'].astype(str)
log_hour = pd.get_dummies(log['HOUR'], prefix='HOUR')
log_hour = pd.concat([log['USRID'], log_hour], axis=1, copy=False)
log_hour = log_hour.groupby('USRID', as_index=False).sum()
data = pd.merge(data, log_hour, on='USRID', how='left')

In [17]:
log_hour.columns

Index(['USRID', 'HOUR_0', 'HOUR_1', 'HOUR_10', 'HOUR_11', 'HOUR_12', 'HOUR_13',
       'HOUR_14', 'HOUR_15', 'HOUR_16', 'HOUR_17', 'HOUR_18', 'HOUR_19',
       'HOUR_2', 'HOUR_20', 'HOUR_21', 'HOUR_22', 'HOUR_23', 'HOUR_3',
       'HOUR_4', 'HOUR_5', 'HOUR_6', 'HOUR_7', 'HOUR_8', 'HOUR_9'],
      dtype='object')

In [18]:
print(data.columns)

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'USRID',
       'FLAG', 'OPR-NUM', 'OPR-NUM-MAX', 'OPR-NUM-MIN', 'OPR-NUM-MEA',
       'OPR-NUM-STD', 'H5_NUM', 'H5_FRQ', 'WEK_1', 'WEK_1_RTE', 'WEK_2',
       'WEK_2_RTE', 'WEK_3', 'WEK_3_RTE', 'WEK_4', 'WEK_4_RTE', 'WEK_5',
       'WEK_5_RTE', 'TIC_CLS_0', 'TIC_CLS_10', 'TIC_CLS_102', 'TIC_CLS_139',
       'TIC_CLS_162', 'TIC_CLS_163', 'TIC_CLS_181', 'TIC_CLS_257',
       'TIC_CLS_259', 'TIC_CLS_326', 'TIC_CLS_359', 'TIC_CLS_372',
       'TIC_CLS_38', 'TIC_CLS_396', 'TIC_CLS_438', 'TIC_CLS_460',
       'TIC_CLS_508', 'TIC_CLS_518', 'TIC_CLS_520', 'TIC_CLS_540',
       'TIC_CLS_604', 'HOUR_0', 'HOUR_1', 'HOUR_10', 'HOUR_11', 'HOUR_12',
       'HOUR_13', 'HOUR_14', 'HOUR_15', 'HOUR_16', 'HOUR_17', 'HOUR_18',
       'HOUR_19', 'HOUR_2', 'HOUR_20', 'HOUR_21', 'HOUR_22',

### 6. 点击时间跨度特征

In [19]:
next_time = log.groupby(by=['USRID'], as_index=False)['next_time'].agg({
    'next_time_mean': np.mean,
    'next_time_std': np.std,
    'next_time_max': np.max,
    'next_time_min': np.min
})

In [20]:
next_time.head()

,USRID,next_time_mean,next_time_std,next_time_max,next_time_min
0,2,8995.125000,25265.607094,71524.0,0.0
1,3,13918.692308,78473.735576,837471.0,0.0
2,4,10.941176,17.020101,64.0,0.0
3,5,67460.421053,202051.360194,649940.0,0.0
4,7,22041.040000,120814.312098,959109.0,0.0


In [21]:
data = pd.merge(data, next_time, on='USRID', how='left')

In [22]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,USRID,FLAG,OPR-NUM,OPR-NUM-MAX,OPR-NUM-MIN,OPR-NUM-MEA,OPR-NUM-STD,H5_NUM,H5_FRQ,WEK_1,WEK_1_RTE,WEK_2,WEK_2_RTE,WEK_3,WEK_3_RTE,WEK_4,WEK_4_RTE,WEK_5,WEK_5_RTE,TIC_CLS_0,TIC_CLS_10,TIC_CLS_102,TIC_CLS_139,TIC_CLS_162,TIC_CLS_163,TIC_CLS_181,TIC_CLS_257,TIC_CLS_259,TIC_CLS_326,TIC_CLS_359,TIC_CLS_372,TIC_CLS_38,TIC_CLS_396,TIC_CLS_438,TIC_CLS_460,TIC_CLS_508,TIC_CLS_518,TIC_CLS_520,TIC_CLS_540,TIC_CLS_604,HOUR_0,HOUR_1,HOUR_10,HOUR_11,HOUR_12,HOUR_13,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_2,HOUR_20,HOUR_21,HOUR_22,HOUR_23,HOUR_3,HOUR_4,HOUR_5,HOUR_6,HOUR_7,HOUR_8,HOUR_9,next_time_mean,next_time_std,next_time_max,next_time_min
0,-1.92554,0,-1.26634,1,0,-1.70618,-0.29641,-0.18761,-0.48351,-0.53051,-0.18703,-0.41609,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.42763,-0.30845,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55917,-0.77398,-0.25708,-0.27221,-0.51336,-0.32438,14233,0,468.0,118.0,2.0,58.50,38.026307,0.0,0.0,26.0,0.055556,0.0,0.000000,0.0,0.000000,0.0,0.000000,442.0,0.944444,44.0,1.0,2.0,24.0,0.0,3.0,0.0,7.0,7.0,296.0,6.0,0.0,11.0,14.0,1.0,0.0,0.0,18.0,34.0,0.0,0.0,0.0,0.0,67.0,2.0,29.0,54.0,30.0,38.0,26.0,5.0,0.0,24.0,74.0,16.0,4.0,24.0,0.0,24.0,0.0,0.0,0.0,0.0,30.0,21.0,5450.297645,92116.141392,1982990.0,0.0
1,-1.92554,0,-1.26634,1,0,-1.70618,0.37601,-0.10030,-0.16694,-0.31897,0.13488,-0.41609,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.49415,-0.43058,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.50058,-0.77398,-0.25708,-0.37907,-0.52723,-0.32438,790,0,243.0,75.0,4.0,20.25,18.805826,0.0,0.0,0.0,0.000000,0.0,0.000000,134.0,0.551440,63.0,0.259259,46.0,0.189300,27.0,0.0,0.0,4.0,0.0,0.0,0.0,27.0,16.0,100.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,55.0,1.0,0.0,22.0,14.0,10.0,13.0,0.0,10.0,19.0,31.0,0.0,0.0,4.0,25.0,13.0,0.0,22.0,9.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,5079.268595,22821.465429,215484.0,0.0
2,-1.92554,0,-1.26634,1,0,-1.57176,0.03980,-0.17089,-0.38610,-0.46000,-0.36669,-0.17962,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.57339,-0.45064,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55127,-0.77398,-0.25708,-0.57113,-0.54758,-0.32438,82308,0,195.0,47.0,4.0,13.00,11.420533,0.0,0.0,84.0,0.430769,47.0,0.241026,60.0,0.307692,0.0,0.000000,4.0,0.020513,19.0,1.0,0.0,5.0,0.0,3.0,0.0,19.0,8.0,14.0,2.0,10.0,0.0,0.0,3.0,0.0,0.0,0.0,111.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,17.0,12.0,14.0,0.0,39.0,4.0,47.0,0.0,8.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,11661.716495,66993.601072,850035.0,0.0
3,-1.92554,0,-1.26634,1,0,-1.30291,-0.29641,-0.18761,-0.45916,-0.53051,-0.11165,-0.35697,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.45451,-0.17717,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.26154,-0.77398,0.23440,-0.32873,-0.51951,-0.32438,90325,0,126.0,29.0,6.0,14.00,7.810250,0.0,0.0,37.0,0.293651,79.0,0.626984,10.0,0.079365,0.0,0.000000,0.0,0.000000,14.0,0.0,0.0,4.0,2.0,3.0,0.0,14.0,8.0,35.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,7.0,12.0,1.0,32.0,0.0,5.0,6.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,30.0,0.0,15.0,8429.248000,38009.474493,350773.0,0.0
4,-1.92554,0,-1.26634,1,0,-1.16849,-0.29641,-0.18761,-0.50786,-0.60103,-0.36994,-0.53433,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.57729,-0.45963,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55917,-0.77398,-0.25708,-0.57910,-0.54884,-0.32438,71893,0,4.0,2.0,2.0,2.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,2.0,0.500000,2.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,75527.333333,130814.580618,226579.0,1.0


In [23]:
log.shape, tic_sum.shape, data.shape

((4425232, 23), (100000, 22), (100000, 98))

### 7.周一至周日，每一天操作的次数

In [24]:
log['DOW'] = log['DOW'].astype(str)
dow_dummy = pd.get_dummies(log['DOW'], prefix='DOW')
data_dow = pd.concat([log['USRID'], dow_dummy], axis=1, copy=False)
data_dow = data_dow.groupby('USRID', as_index=False).sum()
data = pd.merge(data, data_dow, on='USRID', how='left')
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       ...
       'next_time_std', 'next_time_max', 'next_time_min', 'DOW_1', 'DOW_2',
       'DOW_3', 'DOW_4', 'DOW_5', 'DOW_6', 'DOW_7'],
      dtype='object', length=105)

### 8. V30分类特征（没有用，直接drop）

In [25]:
# data = data.drop(['V30'], axis=1)

In [26]:
data.columns, data.shape

(Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
        ...
        'next_time_std', 'next_time_max', 'next_time_min', 'DOW_1', 'DOW_2',
        'DOW_3', 'DOW_4', 'DOW_5', 'DOW_6', 'DOW_7'],
       dtype='object', length=105), (100000, 105))

### 13.V1

In [27]:
# v1_99 = data['V1'].quantile(0.99)
# def process_v1(x):
#     if x > v1_99:
#         x = None
#     return x

In [28]:
# data['V1'] = data['V1'].apply(lambda x: process_v1(x))

In [29]:
# v8_99 = data['V8'].quantile(0.99)
# def process_v1(x):
#     if x > v1_99:
#         x = None
#     return x

# # data['V8'] = data['V8'].apply(lambda x: process_v1(x))

### 14. 每个用户点击优惠券的数字特征

In [30]:
user_ticket = log.groupby(by=['USRID', 'EVT_LBL_1'], as_index=False)['OPR-NUM'].sum()
evt_lbl_1 = user_ticket.groupby(by=['USRID'], as_index=False)['OPR-NUM'].agg({
    'evt1_max': np.max,
    'evt1_min': np.min,
    'evt1_mean': np.mean,
    'evt1_std': np.std
})
data = pd.merge(data, evt_lbl_1, on='USRID', how='left')

### 15.  周一至周日点击量的占总数的比率

In [31]:
data['DOW1_RAT'] = data['DOW_1'] / data['OPR-NUM']
data['DOW2_RAT'] = data['DOW_2'] / data['OPR-NUM']
data['DOW3_RAT'] = data['DOW_3'] / data['OPR-NUM']
data['DOW4_RAT'] = data['DOW_4'] / data['OPR-NUM']
data['DOW5_RAT'] = data['DOW_5'] / data['OPR-NUM']
data['DOW6_RAT'] = data['DOW_6'] / data['OPR-NUM']
data['DOW7_RAT'] = data['DOW_7'] / data['OPR-NUM']

### 16. 点击优惠券的占总数的比例

In [32]:
def num_rate_fea(list):
    for x in list:
        strr = x + '_RAT'
        data[strr] = data[x] / data['OPR-NUM']
        
num_rate_fea(['TIC_CLS_0',
            'TIC_CLS_10',
            'TIC_CLS_102',
            'TIC_CLS_139',
            'TIC_CLS_162',
            'TIC_CLS_163',
            'TIC_CLS_181',
            'TIC_CLS_257',
            'TIC_CLS_259',
            'TIC_CLS_326',
            'TIC_CLS_359',
            'TIC_CLS_372',
            'TIC_CLS_38',
            'TIC_CLS_396',
            'TIC_CLS_438',
            'TIC_CLS_460',
            'TIC_CLS_508',
            'TIC_CLS_518',
            'TIC_CLS_520',
            'TIC_CLS_540',
            'TIC_CLS_604'])

In [33]:
num_rate_fea(['HOUR_0', 'HOUR_1', 'HOUR_10', 'HOUR_11', 'HOUR_12', 'HOUR_13',
       'HOUR_14', 'HOUR_15', 'HOUR_16', 'HOUR_17', 'HOUR_18', 'HOUR_19',
       'HOUR_2', 'HOUR_20', 'HOUR_21', 'HOUR_22', 'HOUR_23', 'HOUR_3',
       'HOUR_4', 'HOUR_5', 'HOUR_6', 'HOUR_7', 'HOUR_8', 'HOUR_9'])

In [34]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,USRID,FLAG,OPR-NUM,OPR-NUM-MAX,OPR-NUM-MIN,OPR-NUM-MEA,OPR-NUM-STD,H5_NUM,H5_FRQ,WEK_1,WEK_1_RTE,WEK_2,WEK_2_RTE,WEK_3,WEK_3_RTE,WEK_4,WEK_4_RTE,WEK_5,WEK_5_RTE,TIC_CLS_0,TIC_CLS_10,TIC_CLS_102,TIC_CLS_139,TIC_CLS_162,TIC_CLS_163,TIC_CLS_181,TIC_CLS_257,TIC_CLS_259,TIC_CLS_326,TIC_CLS_359,TIC_CLS_372,TIC_CLS_38,TIC_CLS_396,TIC_CLS_438,TIC_CLS_460,TIC_CLS_508,TIC_CLS_518,TIC_CLS_520,TIC_CLS_540,TIC_CLS_604,HOUR_0,HOUR_1,HOUR_10,HOUR_11,HOUR_12,HOUR_13,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_2,HOUR_20,HOUR_21,HOUR_22,HOUR_23,HOUR_3,HOUR_4,HOUR_5,HOUR_6,HOUR_7,HOUR_8,HOUR_9,next_time_mean,next_time_std,next_time_max,next_time_min,DOW_1,DOW_2,DOW_3,DOW_4,DOW_5,DOW_6,DOW_7,evt1_max,evt1_min,evt1_mean,evt1_std,DOW1_RAT,DOW2_RAT,DOW3_RAT,DOW4_RAT,DOW5_RAT,DOW6_RAT,DOW7_RAT,TIC_CLS_0_RAT,TIC_CLS_10_RAT,TIC_CLS_102_RAT,TIC_CLS_139_RAT,TIC_CLS_162_RAT,TIC_CLS_163_RAT,TIC_CLS_181_RAT,TIC_CLS_257_RAT,TIC_CLS_259_RAT,TIC_CLS_326_RAT,TIC_CLS_359_RAT,TIC_CLS_372_RAT,TIC_CLS_38_RAT,TIC_CLS_396_RAT,TIC_CLS_438_RAT,TIC_CLS_460_RAT,TIC_CLS_508_RAT,TIC_CLS_518_RAT,TIC_CLS_520_RAT,TIC_CLS_540_RAT,TIC_CLS_604_RAT,HOUR_0_RAT,HOUR_1_RAT,HOUR_10_RAT,HOUR_11_RAT,HOUR_12_RAT,HOUR_13_RAT,HOUR_14_RAT,HOUR_15_RAT,HOUR_16_RAT,HOUR_17_RAT,HOUR_18_RAT,HOUR_19_RAT,HOUR_2_RAT,HOUR_20_RAT,HOUR_21_RAT,HOUR_22_RAT,HOUR_23_RAT,HOUR_3_RAT,HOUR_4_RAT,HOUR_5_RAT,HOUR_6_RAT,HOUR_7_RAT,HOUR_8_RAT,HOUR_9_RAT
0,-1.92554,0,-1.26634,1,0,-1.70618,-0.29641,-0.18761,-0.48351,-0.53051,-0.18703,-0.41609,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.42763,-0.30845,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55917,-0.77398,-0.25708,-0.27221,-0.51336,-0.32438,14233,0,468.0,118.0,2.0,58.50,38.026307,0.0,0.0,26.0,0.055556,0.0,0.000000,0.0,0.000000,0.0,0.000000,442.0,0.944444,44.0,1.0,2.0,24.0,0.0,3.0,0.0,7.0,7.0,296.0,6.0,0.0,11.0,14.0,1.0,0.0,0.0,18.0,34.0,0.0,0.0,0.0,0.0,67.0,2.0,29.0,54.0,30.0,38.0,26.0,5.0,0.0,24.0,74.0,16.0,4.0,24.0,0.0,24.0,0.0,0.0,0.0,0.0,30.0,21.0,5450.297645,92116.141392,1982990.0,0.0,72.0,101.0,49.0,142.0,49.0,55.0,0.0,296.0,1.0,33.428571,76.675855,0.153846,0.215812,0.104701,0.303419,0.104701,0.117521,0.000000,0.094017,0.002137,0.004274,0.051282,0.000000,0.006410,0.0,0.014957,0.014957,0.632479,0.012821,0.000000,0.023504,0.029915,0.002137,0.0,0.0,0.038462,0.072650,0.000000,0.0,0.000000,0.000000,0.143162,0.004274,0.061966,0.115385,0.064103,0.081197,0.055556,0.010684,0.000000,0.051282,0.158120,0.034188,0.008547,0.051282,0.000000,0.051282,0.0,0.0,0.0,0.000000,0.064103,0.044872
1,-1.92554,0,-1.26634,1,0,-1.70618,0.37601,-0.10030,-0.16694,-0.31897,0.13488,-0.41609,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.49415,-0.43058,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.50058,-0.77398,-0.25708,-0.37907,-0.52723,-0.32438,790,0,243.0,75.0,4.0,20.25,18.805826,0.0,0.0,0.0,0.000000,0.0,0.000000,134.0,0.551440,63.0,0.259259,46.0,0.189300,27.0,0.0,0.0,4.0,0.0,0.0,0.0,27.0,16.0,100.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,55.0,1.0,0.0,22.0,14.0,10.0,13.0,0.0,10.0,19.0,31.0,0.0,0.0,4.0,25.0,13.0,0.0,22.0,9.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,5079.268595,22821.465429,215484.0,0.0,18.0,25.0,20.0,42.0,37.0,16.0,85.0,100.0,1.0,27.000000,32.287769,0.074074,0.102881,0.082305,0.172840,0.152263,0.065844,0.349794,0.111111,0.000000,0.000000,0.016461,0.000000,0.000000,0.0,0.111111,0.065844,0.411523,0.049383,0.000000,0.000000,0.004115,0.000000,0.0,0.0,0.000000,0.226337,0.004115,0.0,0.090535,0.057613,0.041152,0.053498,0.000000,0.041152,0.078189,0.127572,0.000000,0.000000,0.016461,0.102881,0.053498,0.000000,0.090535,0.037037,0.049383,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.160494
2,-1.92554,0,-1.26634,1,0,-1.57176,0.03980,-0.17089,-0.38610,-0.46000,-0.36669,-0.17962,-0.43194,-0.13843,-0.27218,-0.188,-0.99764,-0.57339,-0.45064,0.57851,-0.22881,0.15815,-0.16201,0.4762,-0.55127,-0.77398,-0.25708,-0.57113,-0.54758,-0.32438,82308,0,195.0,47.0,4.0,13.00,11.420533,0.0,0.0,84.0,

In [35]:
ticket_df = log[['USRID', 'EVT_LBL_1', 'OCC_TIM_UNI']]

In [36]:
ticket_df = ticket_df.sort_values(by=['USRID', 'EVT_LBL_1', 'OCC_TIM_UNI'])

In [37]:
ticket_df['SAM_TIC_NEX'] = ticket_df.groupby(by=['USRID', 'EVT_LBL_1'])['OCC_TIM_UNI'].diff(1)

In [38]:
ticket_df.head(10)

,USRID,EVT_LBL_1,OCC_TIM_UNI,SAM_TIC_NEX
2,2,0,1.520955e+09,NaN
4,2,326,1.520955e+09,NaN
5,2,326,1.520955e+09,27.0
6,2,326,1.520955e+09,148.0
3,2,359,1.520955e+09,NaN
7,2,359,1.520955e+09,181.0
8,2,359,1.521027e+09,71524.0
0,2,38,1.520955e+09,NaN
1,2,520,1.520955e+09,NaN
14,3,0,1.520042e+09,NaN


In [39]:
ticket_same_max = ticket_df.groupby(by=['USRID', 'EVT_LBL_1'], as_index=False)['SAM_TIC_NEX'].sum()

In [40]:
ticket_same_max_df = ticket_same_max.groupby(by='USRID', as_index=False)['SAM_TIC_NEX'].agg({
    'SAM_TIC_MAX': np.max,
    'SAM_TIC_MIN': np.min,
    'SAM_TIC_MEA': np.mean,
    'SAM_TIC_STD': np.std
})

In [41]:
ticket_same_max_df.head()

,USRID,SAM_TIC_MAX,SAM_TIC_MIN,SAM_TIC_MEA,SAM_TIC_STD
0,2,71705.0,0.0,1.437600e+04,3.204797e+04
1,3,2171314.0,9.0,1.319161e+06,1.019774e+06
2,4,184.0,0.0,7.240000e+01,9.916048e+01
3,5,1281557.0,631609.0,8.916240e+05,3.559576e+05
4,7,1653066.0,7185.0,1.182792e+06,8.030924e+05


In [42]:
data = pd.merge(data, ticket_same_max_df, on='USRID', how='left')

In [43]:
list(data.columns)

['V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'USRID',
 'FLAG',
 'OPR-NUM',
 'OPR-NUM-MAX',
 'OPR-NUM-MIN',
 'OPR-NUM-MEA',
 'OPR-NUM-STD',
 'H5_NUM',
 'H5_FRQ',
 'WEK_1',
 'WEK_1_RTE',
 'WEK_2',
 'WEK_2_RTE',
 'WEK_3',
 'WEK_3_RTE',
 'WEK_4',
 'WEK_4_RTE',
 'WEK_5',
 'WEK_5_RTE',
 'TIC_CLS_0',
 'TIC_CLS_10',
 'TIC_CLS_102',
 'TIC_CLS_139',
 'TIC_CLS_162',
 'TIC_CLS_163',
 'TIC_CLS_181',
 'TIC_CLS_257',
 'TIC_CLS_259',
 'TIC_CLS_326',
 'TIC_CLS_359',
 'TIC_CLS_372',
 'TIC_CLS_38',
 'TIC_CLS_396',
 'TIC_CLS_438',
 'TIC_CLS_460',
 'TIC_CLS_508',
 'TIC_CLS_518',
 'TIC_CLS_520',
 'TIC_CLS_540',
 'TIC_CLS_604',
 'HOUR_0',
 'HOUR_1',
 'HOUR_10',
 'HOUR_11',
 'HOUR_12',
 'HOUR_13',
 'HOUR_14',
 'HOUR_15',
 'HOUR_16',
 'HOUR_17',
 'HOUR_18',
 'HOUR_19',
 'HOUR_2',
 'HOUR_20',
 'HOUR_21',
 'HOUR_22',
 'HOUR_23',

In [44]:
# data['V7&V8'] = data['V7'] * data['V8']

### 构造点击时间是否节假日特征

In [45]:
### 他一般在周几点击，周一到周日七类，每一类加和 比率
### 他在第一二三四周点击的次数加和 比率

In [46]:
log.columns

Index(['USRID', 'EVT_LBL', 'OCC_TIM', 'TCH_TYP', 'EVT_LBL_1', 'EVT_LBL_2',
       'EVT_LBL_3', 'OCC_TIM_UNI', 'YEAR', 'MONTH', 'DOM', 'HOUR', 'MINUTE',
       'SECOND', 'TIM_STP', 'DOW', 'next_time', 'OPR-NUM', 'WEK_1', 'WEK_2',
       'WEK_3', 'WEK_4', 'WEK_5'],
      dtype='object')

In [47]:
data = data.fillna(0)

In [48]:
log[log['USRID'] == 2]

,USRID,EVT_LBL,OCC_TIM,TCH_TYP,EVT_LBL_1,EVT_LBL_2,EVT_LBL_3,OCC_TIM_UNI,YEAR,MONTH,DOM,HOUR,MINUTE,SECOND,TIM_STP,DOW,next_time,OPR-NUM,WEK_1,WEK_2,WEK_3,WEK_4,WEK_5
0,2,38-115-117,2018-03-13 23:26:19,0,38,115,117,1.520955e+09,2018,3,13,23,26,19,2018-03-13 23:26:19,2,1.0,1,0,0,1,0,0
1,2,520-1836-3640,2018-03-13 23:26:20,0,520,1836,3640,1.520955e+09,2018,3,13,23,26,20,2018-03-13 23:26:20,2,36.0,1,0,0,1,0,0
2,2,0-231-277,2018-03-13 23:26:56,0,0,231,277,1.520955e+09,2018,3,13,23,26,56,2018-03-13 23:26:56,2,219.0,1,0,0,1,0,0
3,2,359-1234-2004,2018-03-13 23:30:35,0,359,1234,2004,1.520955e+09,2018,3,13,23,30,35,2018-03-13 23:30:35,2,0.0,1,0,0,1,0,0
4,2,326-1041-1678,2018-03-13 23:30:35,0,326,1041,1678,1.520955e+09,2018,3,13,23,30,35,2018-03-13 23:30:35,2,27.0,1,0,0,1,0,0
5,2,326-1041-1678,2018-03-13 23:31:02,0,326,1041,1678,1.520955e+09,2018,3,13,23,31,2,2018-03-13 23:31:02,2,148.0,1,0,0,1,0,0
6,2,326-1041-1678,2018-03-13 23:33:30,0,326,1041,1678,1.520955e+09,2018,3,13,23,33,30,2018-03-13 23:33:30,2,6.0,1,0,0,1,0,0
7,2,359-1233-2003,2018-03-13 23:33:36,0,359,1233,2003,1.520955e+09,2018,3,13,23,33,36,2018-03-13 23:33:36,2,71524.0,1,0,0,1,0,0
8,2,359-1234-2004,2018-03-14 19:25:40,0,359,1234,2004,1.521027e+09,2018,3,14,19,25,40,2018-03-14 19:25:40,3,NaN,1,0,0,1,0,0


### 将三组特征组合起来

In [49]:
data.to_csv('./data/data_new.csv', index=0)